In [3]:
import  requests
def events_getter ():
    url = "https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/events"

    while True:
        response = requests.get(url)
        data = response.json()
        yield data

        if 'next' not in response.links:
            break
        url = response.links['next']['url']

In [4]:
all_data = []
pages = events_getter()
for page in pages:
    all_data.extend(page)
len(all_data)

299

In [5]:
from datetime import datetime
def process_events(event):
    result = {}
    result['id'] = event['id']
    result['type'] = event['type']
    result['public'] = event['public']

    parsed_timestamp = datetime.fromisoformat(event['created_at'])
    result['create_at'] = parsed_timestamp.timestamp()

    result['actor__id'] = event['actor']['id']
    result['actor__login'] = event['actor']['login']

    topics = event.get('payload', {}).get('pull_request', {}).get('base', {}).get('topics', [])

    return result, topics


In [ ]:
processed_events = []
processed_topics = []
for event in all_data:
    processed_event, topics = process_events(event)
    processed_events.append(processed_event)
    processed_topics.extend(topics)

print(processed_events[:5])
print(processed_topics[:5])

LOADING DATA INTO DUCKDB

Nếu SQLite là "vị vua" trong thế giới dữ liệu giao dịch (OLTP) cho ứng dụng nhỏ, thì DuckDB được mệnh danh là "SQLite dành cho phân tích dữ liệu" (OLAP).
Dưới đây là những đặc điểm cốt lõi giúp bạn hiểu rõ DuckDB là gì:
1. In-process Database (Cơ sở dữ liệu nhúng)
Giống như SQLite, DuckDB không cần server. Nó không chạy như một dịch vụ riêng biệt (như PostgreSQL hay MySQL). Nó tích hợp trực tiếp vào ứng dụng của bạn (ví dụ: chạy ngay bên trong script Python).
Ưu điểm: Không cần cài đặt server phức tạp, không tốn thời gian kết nối qua mạng, dữ liệu nằm ngay trong bộ nhớ hoặc file cục bộ.
2. Columnar Storage (Lưu trữ dạng cột)
Đây là điểm khác biệt lớn nhất giữa DuckDB và SQLite:
SQLite (Row-based): Lưu dữ liệu theo từng dòng. Rất nhanh khi bạn muốn tìm 1 dòng cụ thể, nhưng chậm khi tính toán trên hàng triệu dòng.
DuckDB (Column-based): Lưu dữ liệu theo từng cột. Khi bạn tính SUM(doanh_thu), DuckDB chỉ đọc đúng cột doanh_thu và bỏ qua các cột khác. Điều này giúp nó cực kỳ nhanh cho các tác vụ phân tích (Analytics).
3. Tối ưu cho OLAP (Xử lý phân tích trực tuyến)
DuckDB được thiết kế để chạy các câu lệnh SQL phức tạp trên các tập dữ liệu lớn (hàng triệu đến hàng trăm triệu dòng) ngay trên máy tính cá nhân. Nó sử dụng công nghệ Vectorized Query Execution (thực thi truy vấn theo vector), giúp tận dụng tối đa sức mạnh của CPU hiện đại.
4. Khả năng "đọc mọi thứ" (Swiss Army Knife)
DuckDB cực kỳ mạnh mẽ trong việc đọc và truy vấn trực tiếp các định dạng file dữ liệu phổ biến mà không cần nạp (import) vào database:
Đọc trực tiếp file Parquet, CSV, JSON.
Đọc dữ liệu trực tiếp từ Pandas DataFrame, Polars, hoặc Arrow.
Có thể kết nối và truy vấn file trên S3, HTTP, hoặc Hugging Face.
5. Tại sao Data Engineer lại yêu thích DuckDB?
Cực kỳ nhanh: Trên máy tính cá nhân, nó có thể xử lý dữ liệu nhanh ngang ngửa hoặc hơn cả các cụm Spark lớn nếu dữ liệu vừa phải (vài GB đến vài chục GB).
Cài đặt trong 1 giây: Chỉ cần pip install duckdb.
SQL chuẩn: Hỗ trợ SQL rất đầy đủ và hiện đại.
Tích hợp sâu với Python: Bạn có thể dùng SQL để truy vấn một biến Pandas DataFrame và trả về kết quả dưới dạng một DataFrame khác.

In [1]:
import duckdb

# create a connection to a DuckDB database
conn = duckdb.connect("github_events.db")

In [7]:
processed_events[0]

{'id': '5794410303',
 'type': 'WatchEvent',
 'public': True,
 'create_at': 1768356489.0,
 'actor__id': 134958168,
 'actor__login': 'ruanroliv'}

In [14]:
# create table
conn.execute("""
CREATE TABLE IF NOT EXISTS github_events (
    id TEXT PRIMARY KEY,
    type TEXT,
    public BOOLEAN,
    created_at DOUBLE,
    actor__id BIGINT,
    actor__login TEXT
);
""")

In [18]:
fattened_data = [
    (
        record["id"],
        record["type"],
        record["public"],
        record["create_at"],
        record["actor__id"],
        record["actor__login"]
    )
    for record in processed_events
]


# insert data into the "github_events" table
# Tại sao dùng ?: Thay vì viết trực tiếp dữ liệu vào câu lệnh SQL, bạn dùng dấu hỏi để thư viện (như duckdb hoặc sqlite) tự động điền dữ liệu từ danh sách Python vào. Cách này giúp bảo mật (chống SQL Injection) và xử lý đúng các kiểu dữ liệu (chuỗi, số, ngày tháng).
# ON CONFLICT (id) DO NOTHING: nếu có id (khóa chính) trùng thì bỏ qua dòng đó
conn.executemany("""
INSERT INTO github_events (id, type, public, created_at, actor__id, actor__login)
VALUES (?, ?, ?, ?, ?, ?)
ON CONFLICT (id) DO NOTHING
""", fattened_data)

In [21]:
df = conn.execute("""SELECT * FROM github_events""").df()
df.head()

,id,type,public,created_at,actor__id,actor__login
0,5794410303,WatchEvent,True,1.768356e+09,134958168,ruanroliv
1,5794259842,WatchEvent,True,1.768356e+09,170858816,joemremoto
2,5794167304,WatchEvent,True,1.768355e+09,246016628,ayrtongallo
3,5794092960,WatchEvent,True,1.768355e+09,249157269,srjain92-apollidon
4,5794089336,WatchEvent,True,1.768355e+09,249780930,Matt-YL


In [22]:
conn.close()

# DYNAMIC SCHEMA MANAGEMENT